# Parallel Impact-T run

This example shows how to set up the Impact-T input files and run script, but to leave the running to the user. 



In [ ]:
from impact import Impact
import os

# Nicer plotting
import matplotlib

%config InlineBackend.figure_format = 'retina'
matplotlib.rcParams["figure.figsize"] = (8, 4)

NUMPROCS = 0  # auto-select

In [ ]:
ifile = "templates/lcls_injector/ImpactT.in"
os.path.exists(ifile)

In [ ]:
# Make Impact object
I = Impact(ifile)

# Change some things
I.header["Np"] = 10000 // 2  # Make smaller problem for speed
I.header["Nx"] = 32 // 2
I.header["Ny"] = 32 // 2
I.header["Nz"] = 32 // 2
I.header["Dt"] = 5e-13
I.stop = 3

# Turn on MPI and set parallel domain
I.numprocs = NUMPROCS

I.path

In [ ]:
# Write input to workdir
I.write_input()

# This will be the working path
I.path

In [ ]:
!cat {I.path}/ImpactT.in

# Run externally 
Parallel jobs often need to be run on special nodes and/or submitted to a queue, invoking the run script:

In [ ]:
I.path

In [ ]:
!cat {I.path}/run

Here we will do a quick run 

In [ ]:
%%time
!cd {I.path};./run >log.txt

In [ ]:
# Save into template for the other examples to use
# OUTPATH = os.path.join(os.path.split(ifile)[0], 'output/')
#!cp {I.path}/fort* {OUTPATH}

# Collect output

In [ ]:
I.verbose = True
# Load output
I.load_output()

In [ ]:
I.plot(ylim=(0, 0.002))

# Alternative: load into new object
If this notebook went out of scope, or the run was done previously, we'd need to load the output into a new object.

In [ ]:
I2 = Impact(f"{I.path}/ImpactT.in", use_temp_dir=False)

In [ ]:
I2.load_output()

In [ ]:
I.plot(ylim=(0, 0.002))

# Archiving
Archiving is the same as in the simple example

In [ ]:
afile = os.path.expandvars("output.h5")
I2.archive(afile)

In [ ]:
# Cleanup
os.remove(afile)